In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def classifier(input_shape, width, kernel_size, pool_size):
    model = Sequential()

    model.add(Convolution3D(width*1, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',
                            input_shape=input_shape))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    #model.add(Dropout(0.2))
    model.add(Convolution3D(width*2, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    #model.add(Dropout(0.2))
    model.add(Convolution3D(width*4, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.2))


    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
          #optimizer='adadelta',
          optimizer=Adam(lr=1e-4),                  
          metrics=['accuracy'])
    return model


In [3]:
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [4]:
def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            if '.npy' in subset_path[_]:
                file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[0:60]
    file_list_false = get_dirfiles(output_false)[0:60]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)
    
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def valid_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[-15:]
    file_list_false = get_dirfiles(output_false)[-15:]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)

    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
  
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):

    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 30, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = model_20()
        #model = preds3d_dense(48)
        model = classifier((1, 36, 36, 36), 128 ,(3, 3, 3), (2, 2, 2))
    x,y = train_generator(output_true+'12/',output_false+'12/')
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_data=valid_generator(output_true+'12/',output_false+'12/'),verbose=1, callbacks=callbacks, shuffle=True)
    
 
    return model

In [5]:
fenlei_fit('Fenge_36_36_36_0621_4', load_check = False, batch_size=2, epochs=100, check_name = 'Fenge_36_36_36_0621')

100%|██████████| 15/15 [00:00<00:00, 1796.17it/s]


Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - 7s - loss: 1.1276 - acc: 0.5167 - val_loss: 1.1869 - val_acc: 0.5000
Epoch 2/100
120/120 [==============================] - 5s - loss: 0.9336 - acc: 0.5167 - val_loss: 0.6815 - val_acc: 0.5667
Epoch 3/100
120/120 [==============================] - 4s - loss: 0.8835 - acc: 0.5583 - val_loss: 0.7459 - val_acc: 0.5000
Epoch 4/100
120/120 [==============================] - 4s - loss: 0.8374 - acc: 0.4750 - val_loss: 0.7445 - val_acc: 0.4667
Epoch 5/100
120/120 [==============================] - 4s - loss: 0.8561 - acc: 0.5500 - val_loss: 0.6894 - val_acc: 0.4667
Epoch 6/100
120/120 [==============================] - 4s - loss: 0.6802 - acc: 0.6167 - val_loss: 0.7458 - val_acc: 0.5000
Epoch 7/100
120/120 [==============================] - 4s - loss: 0.7943 - acc: 0.5583 - val_loss: 0.6899 - val_acc: 0.6333
Epoch 8/100
120/120 [==============================] - 4s - loss: 0.6859 - acc: 0.5750 

In [6]:
file_list_true = get_dirfiles(output_true+'12/')
file_list_false = get_dirfiles(output_false+'12/')

In [7]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_36_36_36_0621_4.h5')

In [8]:
cc = []
for i in file_list_false[0:800]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

0.0


In [9]:
cc

[array([[ 0.55378079,  0.44621924]], dtype=float32),
 array([[ 0.56943572,  0.43056428]], dtype=float32),
 array([[ 0.42106602,  0.57893401]], dtype=float32),
 array([[ 0.42034233,  0.57965767]], dtype=float32),
 array([[ 0.49834985,  0.50165021]], dtype=float32),
 array([[ 0.49935961,  0.50064039]], dtype=float32),
 array([[ 0.54745078,  0.45254919]], dtype=float32),
 array([[ 0.54693717,  0.45306289]], dtype=float32),
 array([[ 0.45910752,  0.54089242]], dtype=float32),
 array([[ 0.43996769,  0.56003225]], dtype=float32),
 array([[ 0.56696194,  0.43303809]], dtype=float32),
 array([[ 0.55352014,  0.44647986]], dtype=float32),
 array([[ 0.51186824,  0.48813179]], dtype=float32),
 array([[ 0.5165236 ,  0.48347637]], dtype=float32),
 array([[ 0.47909173,  0.52090824]], dtype=float32),
 array([[ 0.47387573,  0.52612424]], dtype=float32),
 array([[ 0.56686366,  0.43313634]], dtype=float32),
 array([[ 0.56773084,  0.43226919]], dtype=float32),
 array([[ 0.56983125,  0.43016875]], dtype=flo

In [10]:
cc = []
for i in file_list_true[0:800]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [11]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

0.0


In [12]:
cc

[array([[ 0.40309134,  0.59690863]], dtype=float32),
 array([[ 0.3538256 ,  0.64617443]], dtype=float32),
 array([[ 0.38276964,  0.61723036]], dtype=float32),
 array([[ 0.40261695,  0.59738302]], dtype=float32),
 array([[ 0.48044276,  0.51955724]], dtype=float32),
 array([[ 0.42967317,  0.57032675]], dtype=float32),
 array([[ 0.51467663,  0.48532334]], dtype=float32),
 array([[ 0.50625467,  0.4937453 ]], dtype=float32),
 array([[ 0.35383281,  0.64616716]], dtype=float32),
 array([[ 0.37710533,  0.62289464]], dtype=float32),
 array([[ 0.47482833,  0.52517164]], dtype=float32),
 array([[ 0.48120776,  0.51879221]], dtype=float32),
 array([[ 0.34959239,  0.65040761]], dtype=float32),
 array([[ 0.34207135,  0.65792865]], dtype=float32),
 array([[ 0.50370282,  0.49629715]], dtype=float32),
 array([[ 0.51442021,  0.48557982]], dtype=float32),
 array([[ 0.32700127,  0.67299873]], dtype=float32),
 array([[ 0.36964968,  0.63035029]], dtype=float32),
 array([[ 0.4000735 ,  0.59992653]], dtype=flo

In [13]:
len(file_list_true)

76

In [14]:
len(file_list_false)

76